In [1]:
from timeit import default_timer as timer
import os
import numpy as np
from datetime import datetime
from glob import glob
import random
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
print('Date:',datetime.today().strftime('%d/%m/%Y %H:%M'))
spark = SparkSession.builder.config("spark.sql.files.ignoreCorruptFiles","true") .config("spark.serializer","org.apache.spark.serializer.KryoSerializer") .config("spark.yarn.appMasterEnv.LANG","en_US.UTF-8") .config("spark.executorEnv.LANG","en_US.UTF-8") .config("spark.driver.extraJavaOptions","-Duser.timezone=UTC") .config("spark.executor.extraJavaOptions","-Duser.timezone=UTC") .config("spark.sql.session.timeZone","UTC") .config("spark.sql.legacy.timeParserPolicy","LEGACY") .config("spark.driver.memory", "100g") .config("spark.executor.memory", "100g") .config("spark.driver.cores", "32") .config("spark.executor.cores", "32") .config("spark.sql.broadcastTimeout", "-1") .config("spark.sql.autoBroadcastJoinThreshold", "-1") .getOrCreate()
print(spark.sparkContext.getConf().getAll())
path_to_data = os.path.join('/',os.getcwd().split('/')[1],'spf248','twitter_data_collection','data')
print(path_to_data)

Date: 25/05/2023 13:37


23/05/25 17:37:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/25 17:37:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


[('spark.sql.broadcastTimeout', '-1'), ('spark.sql.legacy.timeParserPolicy', 'LEGACY'), ('spark.executor.extraJavaOptions', '-Duser.timezone=UTC'), ('spark.driver.cores', '32'), ('spark.app.id', 'local-1685036260267'), ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), ('spark.serializer.objectStreamReset', '100'), ('spark.executor.cores', '32'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.sql.autoBroadcastJoinThreshold', '-1'), ('spark.driver.port', '41311'), ('spark.ui.reverseProxy', 'true'), ('spark.driver.host', 'cs323.hpc.nyu.edu'), ('spark.ui.proxyBase', '/rnode/cs323.hpc.nyu.edu/8080'), ('spark.executor.memory', '100g'), ('spark.ui.enabled', 'true'), ('spark.executor.id', 'driver'), ('spark.yarn.appMasterEnv.LANG', 'en_US.UTF-8'), ('spark.app.name', 'pyspark-shell'), ('spark.executorEnv.JAVA_HOME', '/usr/lib/jvm/java-11-openjdk-amd64'), ('spark.driver.extraJavaOptions', '-Duser.timezone=UTC'), ('spark.sql.warehouse.dir', 'file:/s

In [3]:
df1 = spark.read.parquet(os.path.join(path_to_data,"user_timeline","batch"))

In [4]:
l = sorted(glob(os.path.join(path_to_data,"user_timeline","API","MX","*2023","update","*.json.bz2")))
random.shuffle(l)
schema = spark.read.option("compression","bzip2").option("multiLine","true").option("encoding","UTF-8").json(l[0]).schema
df2 = spark.read.option("compression","bzip2").option("multiLine","true").option("encoding","UTF-8").option("badRecordsPath", os.path.join(path_to_data,"data_collection","user_timeline","badRecords")).json(l,schema=schema)

23/05/25 17:38:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
start = timer()
tmp = df1.join(df2.selectExpr('user.id_str as user_id'),on='user_id',how='left_anti')
tmp.write.mode("overwrite").parquet(os.path.join(path_to_data,"user_timeline","batch_tmp"))
print('Computing Time:', round(timer() - start), 'sec')

Computing Time: 3397 sec


In [8]:
tmp = spark.read.parquet(os.path.join(path_to_data,"user_timeline","batch_tmp"))

In [9]:
tmp.show()

+-------------------+-------------+-------------------+-------------------+-------+
|            user_id|country_short|           tweet_id|    tweet_timestamp|is_seed|
+-------------------+-------------+-------------------+-------------------+-------+
|1000001503522934784|           PT|               null|               null|      1|
|1000002384779776000|           TR|               null|               null|      1|
|1000003469833572352|           CL|               null|               null|      0|
|1000007918153097216|           EG|               null|               null|      1|
|1000008648695930880|           US|1385743449891033089|2021-04-23 23:52:38|      0|
|1000011054448041984|           DE|               null|               null|      0|
|1000012660694573056|           US|1002389726488350725|2018-06-01 03:21:48|      1|
|          100001956|           US| 110890984828702720|2011-09-06 01:44:05|      1|
|1000025597081522177|           PT|               null|               null| 